<a href="https://colab.research.google.com/github/emersonnieto/Cadastro/blob/main/Treinamento_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df=pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/nlp/master/base_mercadologica.csv')
df.head(2)

,descricao,departamento
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA


In [11]:
df['nova_descricao'] = df['descricao'].copy()
df.head(3)

,descricao,departamento,nova_descricao
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT"
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C/1
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S/GAS 500ML


In [14]:
df['nova_descricao'] = df['nova_descricao'].str.replace('[,.:;!?]+', ' ', regex=True).copy()


In [ ]:
df['nova_descricao'] = df['nova_descricao'].str.replace ('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
df.head(3)

In [ ]:
df['nova_descricao'] = df['nova_descricao'].str.replace('[0-9]+', '', regex=True).copy()
df.head(3)

In [23]:
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct',
              'und','das','no','ou','pc','gr','pt','cm','vd','com','sem','gfa',
              'jg','la','1','2','3','4','5','6','7','8','9','0','a','b','c','d',
              'e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u',
              'v','x','w','y','z']

In [24]:
frase ='Texto exemplo tokenização'
nltk.word_tokenize(frase)

['Texto', 'exemplo', 'tokenização']

In [25]:
cvt = CountVectorizer(strip_accents='ascii',
                      lowercase=True,
                      stop_words=stop_words)

In [26]:
exemplo_descricao = df['nova_descricao'][:2]
exemplo_descricao.values

array(['PASTA INT VITAPOWER  KG AMEND SHOT',
       'ESPONJA BETTANIN BRILHUS C '], dtype=object)

In [28]:
exemplo_descricao_cvt = cvt.fit_transform(exemplo_descricao)
exemplo_descricao_cvt.toarray()


array([[1, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 0, 0]])

In [29]:
X_cvt = cvt.fit_transform(df['nova_descricao'])

In [30]:
tfi = TfidfTransformer(use_idf=True)

In [31]:
exemplo_descricao_tfi = tfi.fit_transform(exemplo_descricao_cvt)
exemplo_descricao_tfi.toarray()

array([[0.4472136 , 0.        , 0.        , 0.        , 0.4472136 ,
        0.4472136 , 0.4472136 , 0.4472136 ],
       [0.        , 0.57735027, 0.57735027, 0.57735027, 0.        ,
        0.        , 0.        , 0.        ]])

In [32]:
X_tfi = tfi.fit_transform(X_cvt)

In [33]:
entrada = X_tfi
saida = df['departamento']
X_train, X_test, y_train, y_test = train_test_split(entrada, saida, test_size=0.2)

In [34]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [36]:
resultado = clf.predict(X_test)
print('Acurácia {:.2f}' .format(metrics.accuracy_score(y_test, resultado)))

Acurácia 0.94


In [39]:
def novo_item(descricao):
  novo_cvt = cvt.transform(pd.Series(descricao))
  novo_tfi = tfi.transform(novo_cvt)
  departamento = clf.predict(novo_tfi)[0]
  return departamento

In [40]:
itens = ['FEIJÃO','ACÚCAR','QUEIJO']
for item in itens:
  print('Produto:', item, 'Departamento:', novo_item(item))

Produto: FEIJÃO Departamento: MERCEARIA SALGADA
Produto: ACÚCAR Departamento: MERCEARIA DOCE
Produto: QUEIJO Departamento: FRIOS
